### Minimal Drawing App

Mit der Maus soll eine Linie gezeichnet werden können, vom Punkt A, wo die
linke Maustaste gedrückt wurde, bis zum Punkt B, wo die linke Maustaste losgelassen wird.

**Tastenbefehle**:  
1. Falls Maustaste nicht gedrückt:
   - c: clear
   - 1-9: Linienbreite setzen
   - Kleinbuchstabe: Strichfarbe setzen
   - Grossbuchstabe: Füllfarbe setzen

<br>  

2. Falls Maustaste gedrückt:
   - ESC: Abbruch (beim Loslassen der Maustaste wird nichts gezeichnet)
   - r/R: stroke/fill Rechteck, A und B sind gegenüberliegende Ecken 
   - c/C: stroke/fill Kreis mit Zentrum A durch B

### Implementation
Der Dictionary `colors` speichert die Farben und der
Dictionary `state` speichert die Position, wo die Maustaste gedrückt wurde und
welche Taste zuletzt gedrückt wurde. 

- Wird die Maustaste gedrückt, wird die Mausposition `(x, y)` in
`state['mouse_down']` gespeichert.
Zudem wird ein Punkt in der Füllfarbe mit Radius linewidth/2 gezeichnet, umrandet von einem Kreis in der Strichfarbe.
- Wird eine Taste gedrückt wird diese in `state['key_pressed']` gespeichert,
  falls die Maustaste gedrückt ist, sonst wird der entsprechende Befehl ausgeführt.
- Wird die Maustaste losgelassen, so
  wird abhängig von der gespeicherten Taste etwas gezeichnet. Dann werden
  `state['mouse_down']` und `state['key_pressed']` auf `None` gesetzt.

In [9]:
from ipywidgets import Output


state = {'mouse_down': None,
         'key_pressed': None,
         }

colors = {'r': 'red',  'o': 'orange', 'b': 'blue', 'g': 'green',
          'y': 'yellow', 'k': 'black', 'p': 'pink', 't': 'teal'}


err_out = Output(layout={'border': '1px solid black'})


def set_color(key):
    if key.isupper():
        color = colors[key.lower()]
        fg.fill_style = color
        bg.fill_style = color
    else:
        fg.stroke_style = colors[key]
        bg.stroke_style = colors[key]


@err_out.capture()
def on_mouse_down(x, y):
    state['mouse_down'] = (x, y)
    fg.fill_circle(x, y, radius=bg.line_width/2)
    fg.stroke_circle(x, y, radius=bg.line_width/2 + 3)


@err_out.capture()
def on_mouse_up(x, y):
    x0, y0 = state['mouse_down']
    state['mouse_down'] = None
    width = x - x0
    height = y - y0
    radius = (width**2+height**2)**.5

    if state['key_pressed'] == 'Escape':
        pass
    elif state['key_pressed'] == 'R':
        bg.fill_rect(x0, y0, width, height)
    elif state['key_pressed'] == 'r':
        bg.stroke_rect(x0, y0, width, height)
    elif state['key_pressed'] == 'C':
        bg.fill_circle(x0, y0, radius)
    elif state['key_pressed'] == 'c':
        bg.stroke_circle(x0, y0, radius)
    else:
        bg.stroke_lines([(x0, y0), (x, y)])

    state['key_pressed'] = None
    fg.clear()


@err_out.capture()
def on_key_down(key, *flags):
    if state['mouse_down']:
        state['key_pressed'] = key
        return

    if key == 'c':
        bg.clear()
    elif key.lower() in colors:
        set_color(key)
    elif key in '123456789':
        bg.line_width = int(key)

In [10]:
from ipycanvas import MultiCanvas
from IPython.display import display

layout = {'border': '1px solid black'}
mcanvas = MultiCanvas(2, width=200, height=200, layout=layout)
bg, fg = mcanvas

mcanvas.on_mouse_down(on_mouse_down)
mcanvas.on_mouse_up(on_mouse_up)
mcanvas.on_key_down(on_key_down)

display(mcanvas, err_out)

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…